In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#%%capture
#!wget -N https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip -o /content/dogImages.zip
#!mv /content/dogImages /content/drive/My\ Drive/dog-breeds/dog_images

# Configuration & utilities

Here we define the configuration parameters, dependencies and utility functions needed for the experiment. You may only need to specify the 'experiment_description' value in the first line of code, and the paths identifying the project and the different datasets. 

In [ ]:
import numpy as np
from glob import glob
import sys
from utils.preprocess import *
from utils.resnet_model import *
from utils.train import *
from utils.simple_model import *


experiment_description = 'refactor'
project_path = '/content/drive/My Drive/dog-breeds/'

##################### DO NOT MODIFY ################################

experiment_location = project_path + experiment_description
escaped_exp_location = experiment_location.replace(" ", "\\ ")
!mkdir $escaped_exp_location
model_file = experiment_location + '/model_ ' + experiment_description
output_file = experiment_location + '/' + experiment_description
sys.path.append(project_path)
##################### END: DO NOT MODIFY ###########################

dog_files = np.array(glob(project_path + "dog_images/*/*/*"))
print('There are %d total dog images.' % len(dog_files))
train_path = project_path + 'dog_images/train'
validation_path = project_path +'dog_images/valid'
test_path = project_path +'dog_images/test'


def print_gpu():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
  else:
    print(gpu_info) 

# Experiment

A lot of things here need to be separated:

 - Creation of dataset loaders
 - NN architecture
 - Hyperparameter dictionaries
 - Testing and Rebooting functions
 - Experiment loop
 - Visualization

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import torch
from torchvision import datasets
import torch.optim as optim
from PIL import ImageFile
import matplotlib.pyplot as plt
import datetime

ImageFile.LOAD_TRUNCATED_IMAGES = True

augmentations = [
                 [iaa.Resize((224, 224)), iaa.Crop(percent=(0, 0.45))]
                ]
learning_rates = [
                 #0.001
                 0.0015,
                 #0.0025,
                 #0.0035
                 ]
optimizers = [
             #optim.Adagrad,
             optim.Adam
             ]
models = [
         resnet18(),
         simple_cnn()
         ]
         
paths = {
    'train_path' : train_path,
    'validation_path' : validation_path,
    'test_path' : test_path
}


hyperparameters_dictionary = {
    'augmentations' : augmentations,
    'learning_rates' : learning_rates,
    'epochs' : 100,
    'optimizers' : optimizers,
    'models' : models
}

print_gpu()
run_experiments(paths, hyperparameters_dictionary, model_file)




Wed Apr 22 22:08:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  